# Reading the data 

---

Lately, the _.fits_ data was transformed into _.csv_ and saved in database/csv_files. We are going to import a light curve sample and work with it.

In [66]:
FILE_PATH = '/content/drive/MyDrive/01 - Iniciação Científica/02 - Datasets/csv_files/EN2_STAR_CHR_0101086161_20070516T060226_20071005T074409.csv'

In [67]:
import pandas as pd

data_sample = pd.read_csv(FILE_PATH)
display(data_sample)

,DATE,WHITEFLUX
0,2007-05-16 18:10:55.071642,112626.77
1,2007-05-16 18:19:27.113766,112605.61
2,2007-05-16 18:27:59.155929,112771.50
3,2007-05-16 18:36:31.198092,113113.60
4,2007-05-16 18:45:03.240256,112621.79
...,...,...
23946,2007-10-05 19:08:02.358665,112438.61
23947,2007-10-05 19:16:34.313684,112406.01
23948,2007-10-05 19:25:06.268742,112496.13
23949,2007-10-05 19:33:38.223801,112344.83


In [68]:
import numpy as np

x = data_sample.DATE.to_numpy()
y = data_sample.WHITEFLUX.to_numpy()

# Bessel Transfer Function

---

The transfer function of a Bessel lowpass filter is defined by

$$ H(s) = \frac{\theta_n (0) }{\theta_n (s/ \omega_0)}  \\ $$ 

$$ \theta_n (s) = \sum_{k=0}^{n} a_k s^k $$

$$ a_k = \frac{(2n-k)!}{2^{n-k}k!(n-k)!} \\ $$




eu simplesmente subtraí a média dos dados antes da filtragem e voltei a somar essa média após a filtragem. Não sei se é esse o problema que você está detectando, mas conversaremos com mais detalhes na nossa reunião hoje.

### Preprocessando

In [23]:
from tools import *

In [24]:
media_curve = np.mean(y)  # 112501.16221318526

#y -= media_curve

In [25]:
viz.view_lightcurve(x, y)

### Parametros

In [46]:
order = 2
cutoff_freq = 0.9

### Biblioteca de controle

In [47]:
# !pip install control
from control import *

In [48]:
### Computing ak

from math import factorial

coef = []
i = 0
while i <= order:
  ak = (factorial(2*order - i)) / ( 2**(order - i)*factorial(i)*factorial(order - i) )
  # print(ak)
  coef.append(ak)
  i += 1

print(coef)

[3.0, 3.0, 1.0]


In [49]:
### Computing θn(s)

s = TransferFunction.s
theta_array = []
k = 0
for k in range(order+1):
  theta_n = coef[k] * (s**k)
  theta_array.append(theta_n)
  # numerical_numerator = coef[0]
  # print(theta_n)

print(theta_array[0])
print(theta_array[1])
print(theta_array[2])
#print(theta_array[3])


3
-
1


3 s
---
 1


s^2
---
 1



In [50]:
### Computing H(s)

coef_numerator = theta_array[0]

list_denominator = theta_array[:]

In [51]:
denominator = 0
for item in list_denominator:
  denominator += item

denominator

TransferFunction(array([1., 3., 3.]), array([1.]))

In [52]:
### Preenchendo transfer function

G = coef_numerator / denominator

print(G)
print(type(G))


      3
-------------
s^2 + 3 s + 3

<class 'control.xferfcn.TransferFunction'>


In [53]:
np.real(evalfr(G, 1))

0.42857142857142855

In [54]:
3/(1**2 + 3*1 + 3)

0.42857142857142855

### Aplicando 



In [55]:
import numpy as np

In [56]:
def bessel(array, fourier_transform, cutoff_freq, order):

  # Extracting features from signal
  n_time = len(array)
  D0 = cutoff_freq * n_time
  xc = n_time

  # Creating the bessel transfer function array
  len_filter = len(fourier_transform)
  filter = np.zeros(len_filter)
  i=0

  for i in range(len_filter):
    filter[i] = np.real(evalfr(G, ( np.abs(i-(xc-1.0))/D0 )))

  return filter

In [57]:
fft = np.fft.fft(y)

In [58]:
filtro = bessel(y, fft, cutoff_freq, order)

In [59]:
applied_filter = fft * filtro

In [60]:
filtered = np.real(np.fft.ifft(applied_filter))

### Normalização

In [61]:
print(y.max())
print(y.min())
print(y.mean())

113609.89
111110.516
112501.16221318526


In [62]:
print(filtered.max())
print(filtered.min())
print(filtered.mean())

45322.360349223665
43645.896414613366
44598.13634094421


In [63]:
y.mean() - filtered.mean()

67903.02587224105

In [64]:
filtered += (y.mean() - filtered.mean())

### Viz

In [45]:
viz.view_filter_results(x, y, x, filtered)

# Tools lib

In [69]:
!pip install control

In [70]:
from tools import *

In [82]:
order = 5
cutoff_freq = 0.9

Filter = filters.FrequencyDomainFiltering()
Filter.filter(array=y, filter_technique='bessel', numExpansion=70, cutoff_freq=cutoff_freq, order=order)
y_filtered = Filter.getFiltered
title = f"Bessel filter with Order = {order} and Cutoff frequency = {cutoff_freq}"
y_filtered += (y.mean() - y_filtered.mean())
viz.view_filter_results(x, y, x, y_filtered, title=title)
#viz.view_lightcurve(x, y_filtered, title=title)